## 0. 호출

In [24]:
from matplotlib import pyplot as plt
from tqdm import tqdm
import pandas as pd

In [39]:
train_data = pd.read_csv ("/opt/new/level2_klue-nlp-03/data/no_noise_train.csv")

In [40]:
train_data

,Unnamed: 0,index,id,sentence,subject_entity,object_entity,label,source,subject_type,object_type
0,0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia,ORG,PER
1,1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree,ORG,ORG
2,2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,ORG,ORG
3,3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,ORG,PER
4,4,4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia,ORG,DAT
...,...,...,...,...,...,...,...,...,...,...
30085,30085,32464,32464,KIA타이거즈 외야수 이창진이 롯데백화점 광주점이 시상하는 9월 월간 MVP에 선정...,"{'word': '이창진', 'start_idx': 12, 'end_idx': 14...","{'word': '외야수', 'start_idx': 8, 'end_idx': 10,...",per:title,wikitree,PER,POH
30086,30086,32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia,PER,PER
30087,30087,32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree,ORG,PER
30088,30088,32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia,ORG,ORG


## 1. 동등관계 Label Swap -> 보류

In [ ]:
aug_label = [
'per:title',
'org:alternate_names',
'per:alternate_names',
'per:other_family',
'per:colleagues',
'per:siblings',
'per:spouse'
]

In [ ]:
aug_rows = train_data[train_data['label'].isin(aug_label)]

In [ ]:
aug_rows['subject_entity'], aug_rows['object_entity'] = aug_rows['object_entity'], aug_rows['subject_entity']

In [ ]:
equal_swap_augmented_data = pd.concat([train_data,aug_rows], ignore_index=True)

In [ ]:
equal_swap_augmented_data.to_csv('equal_swap_augmented_data.csv',index=False)

## 2. 대칭관계 Label Swap

In [41]:
aug_label2 = [
'org:top_members/employees',
'per:employee_of',
'per:children',
'per:parents',
'org:founded_by',
'per:product'
]

In [42]:
aug_rows2 = train_data[train_data['label'].isin(aug_label2)]

In [43]:
modified_rows = pd.DataFrame(columns=aug_rows2.columns)
for index, row in aug_rows2.iterrows():
    if row['label'] == 'org:top_members/employees':
        row['label'] = 'per:employee_of'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    elif row['label'] == 'per:employee_of':
        row['label'] = 'org:top_members/employees'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    elif row['label'] == 'per:children':
        row['label'] = 'per:parents'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    elif row['label'] == 'per:parents':
        row['label'] = 'per:children'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    elif row['label'] == 'org:founded_by':
        row['label'] = 'per:product'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    elif row['label'] == 'per:product':
        row['label'] = 'org:founded_by'
        row['subject_entity'], row['object_entity'] = row['object_entity'], row['subject_entity']
    modified_rows = pd.concat([modified_rows, row.to_frame().T])

In [44]:
sym_augmented_data = pd.concat([train_data,modified_rows], ignore_index=True)

In [45]:
sym_augmented_data.to_csv('sym_augmented_data.csv',index=False)

In [46]:
sym_augmented_data

,Unnamed: 0,index,id,sentence,subject_entity,object_entity,label,source,subject_type,object_type
0,0,0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia,ORG,PER
1,1,1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree,ORG,ORG
2,2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,ORG,ORG
3,3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,ORG,PER
4,4,4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia,ORG,DAT
...,...,...,...,...,...,...,...,...,...,...
38176,30073,32451,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...",per:children,wikipedia,PER,PER
38177,30075,32453,32453,1952년 8월에 실시된 제2대 대통령 선거 후 집권당인 자유당의 내분과 함께 대한...,"{'word': '이승만', 'start_idx': 78, 'end_idx': 80...","{'word': '자유당', 'start_idx': 32, 'end_idx': 34...",per:employee_of,wikipedia,ORG,PER
38178,30082,32461,32461,1852년 미국 대통령 선거에서 휘그당은 미국-멕시코 전쟁의 영웅 테일러 장군의 사...,"{'word': '휘그당', 'start_idx': 18, 'end_idx': 20...","{'word': '밀러드 필모어', 'start_idx': 66, 'end_idx'...",org:top_members/employees,wikipedia,PER,ORG
38179,30087,32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree,ORG,PER


## 3. no_relation 타노스

In [48]:
data = pd.read_csv ("/opt/new/level2_klue-nlp-03/augmentation/sym_augmented_data.csv")

In [50]:
no_relation_indices = data[data['label'] == 'no_relation'].index

# "no_relation"인 절반의 샘플을 삭제
half_no_relation_indices = no_relation_indices[:len(no_relation_indices)//2]
half_no_relation = data.drop(half_no_relation_indices)

In [51]:
half_no_relation.to_csv('tanos_sym_augmented.csv',index=False)

In [52]:
half_no_relation

,Unnamed: 0,index,id,sentence,subject_entity,object_entity,label,source,subject_type,object_type
2,2,2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree,ORG,ORG
3,3,3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree,ORG,PER
5,5,5,5,": 유엔, 유럽 의회, 북대서양 조약 기구 (NATO), 국제이주기구, 세계 보건 ...","{'word': '북대서양 조약 기구', 'start_idx': 13, 'end_i...","{'word': 'NATO', 'start_idx': 25, 'end_idx': 2...",org:alternate_names,wikipedia,ORG,ORG
7,7,7,7,"박용오(朴容旿, 1937년 4월 29일(음력 3월 19일)(음력 3월 19일) ~ ...","{'word': '박용오', 'start_idx': 0, 'end_idx': 2, ...","{'word': '1937년 4월 29일', 'start_idx': 9, 'end_...",per:date_of_birth,wikipedia,PER,DAT
8,8,8,8,중공군에게 온전히 대항할 수 없을 정도로 약해진 국민당은 타이베이로 수도를 옮기는 ...,"{'word': '중화민국', 'start_idx': 59, 'end_idx': 6...","{'word': '타이베이', 'start_idx': 32, 'end_idx': 3...",org:place_of_headquarters,wikipedia,ORG,LOC
...,...,...,...,...,...,...,...,...,...,...
38176,30073,32451,32451,"반면 정재학의 3남인 정종원은 제5대 조흥은행장, 손자인 정운용은 제2대 조흥은행장...","{'word': '정재학', 'start_idx': 3, 'end_idx': 5, ...","{'word': '정종원', 'start_idx': 12, 'end_idx': 14...",per:children,wikipedia,PER,PER
38177,30075,32453,32453,1952년 8월에 실시된 제2대 대통령 선거 후 집권당인 자유당의 내분과 함께 대한...,"{'word': '이승만', 'start_idx': 78, 'end_idx': 80...","{'word': '자유당', 'start_idx': 32, 'end_idx': 34...",per:employee_of,wikipedia,ORG,PER
38178,30082,32461,32461,1852년 미국 대통령 선거에서 휘그당은 미국-멕시코 전쟁의 영웅 테일러 장군의 사...,"{'word': '휘그당', 'start_idx': 18, 'end_idx': 20...","{'word': '밀러드 필모어', 'start_idx': 66, 'end_idx'...",org:top_members/employees,wikipedia,PER,ORG
38179,30087,32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...","{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...",per:employee_of,wikitree,ORG,PER


## 4. 숫자 레이블 증강 -> 의미 없음
### 해당 레이블의 정확도는 이미 높은 수준

## 5. 유의어 증강 -> 폐기
### 해당 기법으로는 문장 수의 유의미한 증강 X

In [53]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from konlpy.tag import Kkma
from tqdm import tqdm
import torch
model_name = "klue/roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class SynonymAugment:
  
  def __init__(self,sentence):
    self.sentence = sentence
    self.VA_list = []
    self.masking = []
    self.tokens = []
    self.mask_ids = []
    self.augmented_sentence = []
    self.masked_sentences = []

  # pos태깅 후 형용사(VA)추출
  def VA_find(self):
    kkma = Kkma()
    pos = kkma.pos(self.sentence)
    for pos_tagged in pos:
      if pos_tagged[1] == "VA":
          self.VA_list.append(pos_tagged[0])

  def tokenmaker(self):
      self.tokens=tokenizer.tokenize(self.sentence)
  
  # VA 길이에 따른 토큰 추적
  ## '예쁜' 의 경우 VA->'예쁘' 따라서 '예'를 포함하고 있는 token 추적
  ## '많은' 의 경우 VA->'많' 따라서 '많'을 포함하고 있는 token 추적 
  def masking_set(self):
    for VA in self.VA_list:
      for token in self.tokens:
        if len(VA[0]) != 1:
            if VA[0][:-1] in str(token):
                self.masking.append(token)
        else:
            if VA[0] in str(token):
                self.masking.append(token)

  #단어의 위치를 찾는 경우에만 mask token 부여
    for word in self.masking:
      masks = self.sentence.find(word)
      if masks != -1:
          self.mask_ids.append(masks)


  def masking_sentence(self):
      for mask_idx in range(len(self.mask_ids)):
        self.masked_sentences.append(self.sentence[:self.mask_ids[mask_idx]]
                                + tokenizer.mask_token       
                                + self.sentence[self.mask_ids[mask_idx] + len(self.masking[mask_idx]):])

  def prediction(self):
  # 예측 모델
      for masked_sentence in range(len(self.masked_sentences)):
          input_ids = tokenizer.encode(self.masked_sentences[masked_sentence], return_tensors="pt")
          mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]
          output = model(input_ids.to(device))[0]  
          softmax = torch.nn.Softmax(dim=-1)
          probs = softmax(output[0, mask_token_index, :]).detach().numpy()

  # 예측 결과 출력
          predicted_token = tokenizer.convert_ids_to_tokens(torch.argmax(output[0, mask_token_index, :], dim=1).tolist())[0]
          if predicted_token != "[UNK]" and predicted_token != "[PAD]":
              new_sentence = self.sentence[:self.mask_ids[masked_sentence]]                                           \
                              + predicted_token                                                                       \
                              + self.sentence[self.mask_ids[masked_sentence] + len(self.masking[masked_sentence]):]
              if new_sentence != self.sentence:
                  self.augmented_sentence.append(new_sentence)
        
      return print(self.augmented_sentence)
aug_label = ['org:dissolved','org:members','org:political/religious_affiliation','per:other_family','per:place_of_birth','per:place_of_residene','per:siblings	']
aug_df=pd.read_csv("/opt/new/level2_klue-nlp-03/augmentation/tanos_sym_augmented.csv")
aug_df=aug_df[aug_df["label"].isin(aug_label)]
sentences = aug_df['sentence'].values.tolist()
#augmentation 실행
augmented_list=[]

for i in tqdm(range(len(sentences))):
  augment1 = SynonymAugment(sentences[i])
  augment1.tokenmaker()
  augment1.VA_find()
  augment1.masking_set()
  augment1.masking_sentence()
  augment1.prediction()
  augmented_list.append(augment1.augmented_sentence)


# # 여러가지 후보 중 우선 첫번째 출력된 문장만 교체
# ## 모든 후보를 교체할 수 있게끔 수정 예정
# for j in range(len(sentences)):
#   if aug_list1[j] != []:
#     df["sentence_1"].replace(df["sentence_1"][j],aug_list1[j][0],inplace=True)


df.to_csv('syn_aug_dev.csv', index=False)
count=0
for i in aug_list1:
  if i != []:
    count += 1
for i in aug_list2:
  if i != []:
    count += 1

## 6. Back Translation